In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import librosa
import json

In [ ]:
def create_dataset(folder_path): 
  images = []
  names = []
 
  for file in os.listdir(folder_path):

    image_path = os.path.join(folder_path, file)

    size = Image.open(image_path).size
    image = np.array(Image.open(image_path).resize((int(size[0]/4),int(size[1]/2))))
    image = image.astype('float32')
    image = np.delete(image, [3], axis=2)

    image /= 255.0
    if file[:-4] != 'axqtExFY5-s (1)':
      images.append(image)
      names.append(file[:-4])

  max_len = 0
  for i in range(len(images)):
    length = images[i].shape[1]
    if length>max_len:
      max_len=length

  for i in range(len(images)):
    images[i] = librosa.util.fix_length(images[i], int(max_len), axis=1)

  images = np.array(images)

  df = pd.read_csv('/content/drive/MyDrive/YouTubeID_and_Keywords.csv')

  keywords = []
  for i in names:
    if i in df.YTID.to_list():
      keys = df.Keywords[df.YTID == i].to_list()
      keywords.append(keys[0].split(';'))
    else:
      print(i)

  return names,images,keywords

In [ ]:
names, images, keywords = create_dataset('/content/drive/MyDrive/Pre-Training Images')

In [ ]:
all_words = []
for i in keywords:
  for j in i:
    if len(j)>2:
      all_words.append(j)
      
from collections import Counter

frequency_list = Counter(all_words).most_common(300)

keywords_updated = []
for j in range(len(keywords)):
  keyword_list = []
  for i in frequency_list:
    if i[0] in keywords[j]:
      keyword_list.append(i[0])
  keywords_updated.append(keyword_list)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
multihot = mlb.fit_transform(keywords_updated)
class_labels = list(mlb.classes_)
print(len(class_labels))

300


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, multihot, test_size=0.20, random_state=33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(15100, 64, 107, 3) (3776, 64, 107, 3) (15100, 300) (3776, 300)


In [ ]:
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import SGD
from keras.layers.pooling import AveragePooling2D
from keras.layers.normalization.batch_normalization import BatchNormalization
import tensorflow as tf

In [ ]:
# Create CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(64, 107, 3)))
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(len(class_labels), activation='softmax'))

In [ ]:
# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=SGD(learning_rate=0.0001, momentum=0.6),
    metrics=['accuracy'],
)

In [ ]:
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

Epoch 1/15
472/472 [==============================] - 896s 2s/step - loss: 10.5358 - accuracy: 0.1158 - val_loss: 15.2190 - val_accuracy: 0.0040
Epoch 2/15
472/472 [==============================] - 904s 2s/step - loss: 9.5182 - accuracy: 0.1755 - val_loss: 9.2260 - val_accuracy: 0.1772
Epoch 3/15
472/472 [==============================] - 911s 2s/step - loss: 9.0464 - accuracy: 0.1902 - val_loss: 9.1352 - val_accuracy: 0.1917
Epoch 4/15
472/472 [==============================] - 998s 2s/step - loss: 8.7068 - accuracy: 0.2028 - val_loss: 8.7243 - val_accuracy: 0.2037
Epoch 5/15
472/472 [==============================] - 961s 2s/step - loss: 8.4625 - accuracy: 0.2077 - val_loss: 8.9073 - val_accuracy: 0.2018
Epoch 6/15
472/472 [==============================] - 889s 2s/step - loss: 8.3000 - accuracy: 0.2167 - val_loss: 8.5840 - val_accuracy: 0.2209
Epoch 7/15
472/472 [==============================] - 908s 2s/step - loss: 8.1625 - accuracy: 0.2185 - val_loss: 8.7442 - val_accuracy: 0.19